In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [3]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=2, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [4]:
def calcBc():
    r = 1/Integrate(1,mesh)*Integrate(x,mesh)
    return r
calcBc()

-5.080028269950915e-17

In [5]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [6]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
vol_inv = Parameter(1)
vol_inv.Set(1/Integrate(surf_t,mesh))
# bc prep
bc_tx = CoefficientFunction(x)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)

# bc constraints
beta0 = 1e-3
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_int = Parameter(1)
bc_int.Set(Integrate(bc_tx,mesh))
bc_x.Set(calcBc())

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [7]:
dJOmega.Assemble()
print("curvol: ", vol.Get())
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

curvol:  12.566430839284152
Initial bc:  -5.080028269950915e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [8]:
mesh.UnsetDeformation()
test = CoefficientFunction((0.8,1.2))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [9]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print("curvol: ", Integrate(1,mesh))
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
print(dJOmega.vec)

curvol:  12.56643083928415
0.7999999999999996
0.7999999999999996
  1.89735e-19
 -0.000392056

  0.000392056
 1.35525e-19

  -4.40457e-20
 0.000392056

  -0.000392056
 -8.80914e-20

  0.000123018
 -0.0003812

  0.000235182
 -0.000324147

  0.000324147
 -0.000235182

  0.0003812
 -0.000123018

  0.0003812
 0.000123018

  0.000324147
 0.000235182

  0.000235182
 0.000324147

  0.000123018
 0.0003812

  -0.000123018
 0.0003812

  -0.000235182
 0.000324147

  -0.000324147
 0.000235182

  -0.0003812
 0.000123018

  -0.0003812
 -0.000123018

  -0.000324147
 -0.000235182

  -0.000235182
 -0.000324147

  -0.000123018
 -0.0003812

  6.77626e-21
 -1.0842e-19

  -8.13152e-20
      0

  -1.0842e-19
 -2.71051e-20

  6.94567e-20
 1.0842e-19

       0
 -1.35525e-19

  5.42101e-20
 -2.71051e-20

  -1.35525e-20
 1.0842e-19

  -5.42101e-20
 1.35525e-20

  5.42101e-20
 -8.13152e-20

  2.71051e-20
 -2.71051e-20

  1.0842e-19
 -1.81053e-20

       0
 5.42101e-20

  2.71051e-20
 1.0842e-19

  -5.42101e-20
 -

## try to do this as shape optimization
Barycenter:
$\Large bc = \frac{1}{\int_\Omega 1 \mathrm{dx}}\cdot\int_\Omega x \mathrm{dx}$ <br>
$\Large \frac{\partial }{\partial x} \beta(bc(x)-bc_0)^2 =$ ... <br>
$\Large \frac{\partial bc_{trafo}}{\partial X}= \frac{div(X)\int xdx-\int 1dx\int X_1 dx}{(\int xdx)^2} $ unimplementierbar?!

In [44]:
# don't use 2 deformations, just set bc_0x to something else, let's see if we get there:
mesh.UnsetDeformation()
bc_0x = 3

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
# initial try in auto (wrong)
#dJOmega2 += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)
# derivative by sturm (wrong)
#dJOmega2 += ((2*beta*((1/bc_x-bc_0x))*(1/bc_x)*dx)).DiffShape(X)
# another try (wrong)
# dJOmega2 += (2*(1/(vol_inv*bc_int)-bc_0x)*(1/(vol_inv*bc_int))*dx).DiffShape(X)
# 
#dJOmega2 += 

gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [45]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
bc_x.Get()

-5.080028269950915e-17

In [46]:
import time
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 100

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    #bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    vol_inv.Set(1/Integrate(surf_t,mesh))
    bc_int.Set(Integrate(bc_tx,mesh))
    print(1/Integrate(1,mesh)*Integrate(x,mesh))
    #print(bc_x.Get())
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    
    time.sleep(0.1)

-5.080028269950915e-17
0.004334282701915581
0.017571449417101676
0.01816207956700874
0.02791769207187798
0.01489319747730005
0.02650208460673902
0.034807934212739984
0.019881868926274443
0.03361032270109616
0.04491658815858835


ZeroDivisionError: float division by zero

In [ ]:
bc_x.Get()

In [15]:
a = gfX.MDComponent(1)